In [1]:
# Cell 1: Input Database Path

# Input for database path
oo_db_path = '/home/dimitri/code/oll_onemax/computed/cirrus/iphone.db'

In [2]:
# Cell 2: Process and Display Config Table

import sqlite3
import pandas as pd

def load_and_flatten_config(db_path):
  try:
    with sqlite3.connect(db_path) as conn:
      cursor = conn.cursor()
      cursor.execute("SELECT key, value FROM config")
      rows = cursor.fetchall()

    config = {}
    for key, value in rows:
      if value.isdigit():
        parsed_value = int(value)
      elif all(char.isdigit() or char == '.' for char in value):
        try:
          parsed_value = float(value)
        except ValueError:
          parsed_value = value
      else:
        parsed_value = value

      key_parts = key.split('__')
      d = config
      for part in key_parts[:-1]:
        if part not in d:
          d[part] = {}
        d = d[part]
      d[key_parts[-1]] = parsed_value

    flattened_config = []
    for k, v in config.items():
      flattened_config.extend(flatten_config(v, k))

    return pd.DataFrame(flattened_config)

  except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    return pd.DataFrame()

def flatten_config(config, parent_key=''):
  items = []
  for k, v in config.items():
    new_key = f"{parent_key}__{k}" if parent_key else k
    if isinstance(v, dict):
      items.extend(flatten_config(v, new_key))
    else:
      if isinstance(v, (int, float)):
        formatted_value = f"{v:,}"
      else:
        formatted_value = v
      items.append({'Key': new_key, 'Value': formatted_value})
  return items

# Load, process, and display the config data
flattened_config_df = load_and_flatten_config(oo_db_path)
flattened_config_df

AttributeError: 'int' object has no attribute 'items'